In [ ]:

#
# 
#
# Author: Caleb Carter
# Student Number: 910678
#
# Version: 1.2
#
# This program schedules different RSP bots to play against each
# other.
# The AI bots are first through to fifth order Markov models and
# also dynamically switching between different order Markov models.
#
# Rule based bots have been added to test non AI versus AI solution.
#
#
###############################################################################################

from __future__ import division
import random
import math
import itertools
import operator                        
#

beat = {'R': 'P', 'P': 'S', 'S': 'R'}  #These are the moves that would beat an opponent.

################################################################################################
# Class declarations for Markov Model that is used in a number of different routines
################################################################################################

class MarkovChain():
    def __init__(self, order, decay=1.0):
        self.decay = decay
        self.matrix = self.create_matrix(order)

    @staticmethod
    def create_matrix(order):
#
# First order Markov chain creates keys = ['RR', 'RP', 'RS', 'PR', 'PP', 'PS', 'SR', 'SP', 'SS'] which
# which are all the combinations of the moves that two persons can play with three possible outcomes ie. 3^^2 = 9
#
# Second Order Markov chain creates keys = ['PSPR', 'PSPP', 'PPRS', 'PRSS', 'PRSR', 'PPSP', etc etc] which
# which are all the combinations of the moves that two persons can play with three possible outcomes ie. 3^^4 = 81
#
# etc etc for higher orders.
#
        def create_keys(order):

            keys_initial = ['R', 'P', 'S']
            keys = ['R', 'P', 'S']
            key_initial_len = len(keys_initial)
            keys_final = []
            strip_out=3

            for i in range((order * 2-1)): 
                key_len = len(keys)
                strip_out=strip_out*3
                for k in range(key_len):
                    for j in range(key_initial_len):    #This method of key generation is for higher order Markov chain
                        key_extend=keys[k]+keys_initial[j]
                        keys_final.append(key_extend)
                keys=keys_final
                key_len = len(keys)
            keys = keys[-1*strip_out:]
            return keys
#
        keys = create_keys(order)
#        print(keys)
        matrix = {}
        for key in keys:
            matrix[key] = {'R': {'prob': 1 / 3,                       
                                 'n_obs': 0
                                 },
                           'P': {'prob': 1 / 3,                                                               
                                 'n_obs': 0
                                 },
                           'S': {'prob': 1 / 3,                      
                                 'n_obs': 0
                                 }
                           }
#
# Data structure of First Order Markov Model.
#
# This creates or initializes the probability mapping matrix. All possible combinations of two players ([n-1] game can be mapped to a single output which gives
# our bots output for nth game. 
# print(matrix)
#        {'PR': {'P': {'n_obs': 0, 'prob': 0.3333333333333333}, 'S': {'n_obs': 0, 'prob': 0.3333333333333333}, 'R': {'n_obs': 0, 'prob': 0.3333333333333333}},
#         'PS': {'P': {'n_obs': 0, 'prob': 0.3333333333333333}, 'S': {'n_obs': 0, 'prob': 0.3333333333333333}, 'R': {'n_obs': 0, 'prob': 0.3333333333333333}},
#         'PP': {'P': {'n_obs': 0, 'prob': 0.3333333333333333}, 'S': {'n_obs': 0, 'prob': 0.3333333333333333}, 'R': {'n_obs': 0, 'prob': 0.3333333333333333}},
#         'RP': {'P': {'n_obs': 0, 'prob': 0.3333333333333333}, 'S': {'n_obs': 0, 'prob': 0.3333333333333333}, 'R': {'n_obs': 0, 'prob': 0.3333333333333333}},
#         'RR': {'P': {'n_obs': 0, 'prob': 0.3333333333333333}, 'S': {'n_obs': 0, 'prob': 0.3333333333333333}, 'R': {'n_obs': 0, 'prob': 0.3333333333333333}},
#         'RS': {'P': {'n_obs': 0, 'prob': 0.3333333333333333}, 'S': {'n_obs': 0, 'prob': 0.3333333333333333}, 'R': {'n_obs': 0, 'prob': 0.3333333333333333}},
#         'SS': {'P': {'n_obs': 0, 'prob': 0.3333333333333333}, 'S': {'n_obs': 0, 'prob': 0.3333333333333333}, 'R': {'n_obs': 0, 'prob': 0.3333333333333333}},
#         'SR': {'P': {'n_obs': 0, 'prob': 0.3333333333333333}, 'S': {'n_obs': 0, 'prob': 0.3333333333333333}, 'R': {'n_obs': 0, 'prob': 0.3333333333333333}},
#         'SP': {'P': {'n_obs': 0, 'prob': 0.3333333333333333}, 'S': {'n_obs': 0, 'prob': 0.3333333333333333}, 'R': {'n_obs': 0, 'prob': 0.3333333333333333}}}
#
# Data structure of Second Order Markov Model
# 
# This creates or initializes the probability mapping matrix. All possible combinations of two players ([n-1] and [n-2] games can be mapped to a single output
# which gives our bots output for nth game.
# print(matrix)
#       {'SPPP': {'P': {'n_obs': 0, 'prob': 0.3333333333333333}, 'S': {'n_obs': 0, 'prob': 0.3333333333333333}, 'R': {'n_obs': 0, 'prob': 0.3333333333333333}},
#        'PPRS': {'P': {'n_obs': 0, 'prob': 0.3333333333333333}, 'S': {'n_obs': 0, 'prob': 0.3333333333333333}, 'R': {'n_obs': 0, 'prob': 0.3333333333333333}},
#        'PRSS': {'P': {'n_obs': 0, 'prob': 0.3333333333333333}, 'S': {'n_obs': 0, 'prob': 0.3333333333333333}, 'R': {'n_obs': 0, 'prob': 0.3333333333333333}},
#        'PRSR': {'P': {'n_obs': 0, 'prob': 0.3333333333333333}, 'S': {'n_obs': 0, 'prob': 0.3333333333333333}, 'R': {'n_obs': 0, 'prob': 0.3333333333333333}},
#         .......
#        'SPPR': {'P': {'n_obs': 0, 'prob': 0.3333333333333333}, 'S': {'n_obs': 0, 'prob': 0.3333333333333333}, 'R': {'n_obs': 0, 'prob': 0.3333333333333333}},
#        'SRRP': {'P': {'n_obs': 0, 'prob': 0.3333333333333333}, 'S': {'n_obs': 0, 'prob': 0.3333333333333333}, 'R': {'n_obs': 0, 'prob': 0.3333333333333333}},
#        'SSPR': {'P': {'n_obs': 0, 'prob': 0.3333333333333333}, 'S': {'n_obs': 0, 'prob': 0.3333333333333333}, 'R': {'n_obs': 0, 'prob': 0.3333333333333333}},
#        'SSPS': {'P': {'n_obs': 0, 'prob': 0.3333333333333333}, 'S': {'n_obs': 0, 'prob': 0.3333333333333333}, 'R': {'n_obs': 0, 'prob': 0.3333333333333333}},
# etc up to 81 elements in the matrix}
#
# etc etc for higher order Markov models....
#
# This is ike a simple database where the key into the database are the moves by two opposing bots dependant on order.
#
#        print(matrix)
        return matrix
#
# This routine updates the Markov Chains probability matrix
#
    def update_matrix(self, pair, input_bot):
        for i in self.matrix[pair]:
#            print("before mult ",self.matrix[pair][i]['n_obs'])
            self.matrix[pair][i]['n_obs'] = self.decay * self.matrix[pair][i]['n_obs']
#            print("after mult ",self.matrix[pair][i]['n_obs'])
        self.matrix[pair][input_bot]['n_obs'] = self.matrix[pair][input_bot]['n_obs'] + 1
#        print("decay ",self.decay)
#        print("[n-2] game moves ",pair)
#        print("[n-1] opposition move ",input_bot)
#        print("n_obs [n-1] of opposition moves and [n-2] game moves subject to decay",self.matrix[pair][input_bot]['n_obs'])
        n_total = 0
        for i in self.matrix[pair]:
            n_total += self.matrix[pair][i]['n_obs']

        for i in self.matrix[pair]:
#            print(i)
#            print(n_total)
            self.matrix[pair][i]['prob'] = self.matrix[pair][i]['n_obs'] / n_total
#            print("Prob of R, P, S",self.matrix[pair][i]['prob'])
#        print(self.matrix)


    def predict(self, pair):
#
# The logic is by knowing the moves of the last [n-1] etc games for both bots you are predicting what the likely move is going to be
# by the competitor bot for the [nth] game by building up the 'prob' and 'n_obs' values based on their past stratagies. The n_obs has
# a 'decay' factor so that if a person changes strategy your bot is not held back by the competitor bot's changing strategy.
#
        probs = self.matrix[pair]
# This filters from the 9 by 3 (First Order), 81 by 3 (Second Order) etc (higher order) matrix the 'prob' and 'n_obs' values for
# 'R' 'S' 'P' for the [n-1] game
        probs_array = ((probs['P']['prob'])+(probs['P']['n_obs']),(probs['S']['prob'])+(probs['S']['n_obs']),(probs['R']['prob'])+(probs['R']['n_obs']))
#
        if max(probs_array) == min(probs_array):  #If 'prob' and 'n_obs' values for 'R' 'S' 'P' still the same as initialized values then make a random choice.
            return random.choice(['R', 'P', 'S'])
        else:
            if (probs_array.index(max(probs_array))) == 0: #The array row is searched to find the highest n_obs and prob for the [n-1] game between 'R', 'P', and 'S'
                return 'P'
            elif (probs_array.index(max(probs_array))) == 1:
                return 'S'
            else:
                return 'R'

class RandomPredictor():
# A random predictor that gives a random output to initialize the game.
    @staticmethod
    def predict():
        return random.choice(['R','P','S'])
#
# An example of first order Markov Model probability mapping matrix when played for a few games against a bot that
# simply puts out 'R','R' continuously
#
# The output from the last iteration of the 'update matrix' function call:
#
# {'PR': {'P': {'n_obs': 0.0, 'prob': 0.0}, 'S': {'n_obs': 0.0, 'prob': 0.0}, 'R': {'n_obs': 5.217031, 'prob': 1.0}},
#  'PS': {'P': {'n_obs': 0, 'prob': 0.3333333333333333}, 'S': {'n_obs': 0, 'prob': 0.3333333333333333}, 'R': {'n_obs': 0, 'prob': 0.3333333333333333}}, 
#  'PP': {'P': {'n_obs': 0, 'prob': 0.3333333333333333}, 'S': {'n_obs': 0, 'prob': 0.3333333333333333}, 'R': {'n_obs': 0, 'prob': 0.3333333333333333}}, 
#  'RP': {'P': {'n_obs': 0, 'prob': 0.3333333333333333}, 'S': {'n_obs': 0, 'prob': 0.3333333333333333}, 'R': {'n_obs': 0, 'prob': 0.3333333333333333}}, 
#  'RR': {'P': {'n_obs': 0, 'prob': 0.3333333333333333}, 'S': {'n_obs': 0, 'prob': 0.3333333333333333}, 'R': {'n_obs': 0, 'prob': 0.3333333333333333}}, 
#  'RS': {'P': {'n_obs': 0, 'prob': 0.3333333333333333}, 'S': {'n_obs': 0, 'prob': 0.3333333333333333}, 'R': {'n_obs': 0, 'prob': 0.3333333333333333}}, 
#  'SS': {'P': {'n_obs': 0, 'prob': 0.3333333333333333}, 'S': {'n_obs': 0, 'prob': 0.3333333333333333}, 'R': {'n_obs': 0, 'prob': 0.3333333333333333}}, 
#  'SR': {'P': {'n_obs': 0, 'prob': 0.3333333333333333}, 'S': {'n_obs': 0, 'prob': 0.3333333333333333}, 'R': {'n_obs': 0, 'prob': 0.3333333333333333}}, 
#  'SP': {'P': {'n_obs': 0, 'prob': 0.3333333333333333}, 'S': {'n_obs': 0, 'prob': 0.3333333333333333}, 'R': {'n_obs': 0, 'prob': 0.3333333333333333}}}
#
# The array is then searched to find the highest n_obs and prob for the [n-1] game between 'R', 'P', and 'S'
#
# Thus the 'predict function' gives: 'R'
#
# Then the 'beat function' is to beat the predicted value which then gives: 'P'
#
#
############################################################################
# LIST OF ALL THE BOTS ROUTINES.
############################################################################

############################################################################
# This bot uses a simple randomized rule to decide which option to return.
############################################################################
#
def RuleOutput(input_bot):
    global rockCount,paperCount,scissorsCount
    if input_bot == "": # initialize variables for the first round
            rockCount = paperCount = scissorsCount = 0
    elif input_bot == "R":
            rockCount += 1
    elif input_bot == "P":
            paperCount += 1
    elif input_bot == "S":
            scissorsCount += 1
    if rockCount > paperCount and rockCount > scissorsCount:
            output = "P" # Paper beats Rock
    elif paperCount > scissorsCount:
            output = "S" # Scissors beats Paper
    else:
            output = "R" # Rock beats Scissors
    return output


#
########################################################################
# This bot uses a fixed rule.
#######################################################################
#
def FixedRuleOutput(input_bot):
    global output_Rule_prev
    if input_bot == "": # initialize variables for the first round
            output=output_Rule_prev=random.choice(['R', 'P', 'S'])
    elif input_bot == "R" and output_Rule_prev == "S":
            output = output_Rule_prev = "R"
    elif input_bot == "R" and output_Rule_prev == "P":
            output = output_Rule_prev = "P"
    elif input_bot == "R" and output_Rule_prev == "R":
            output =output_Rule_prev = "S"
    #
    elif input_bot == "S" and output_Rule_prev == "S":
            output = output_Rule_prev ="P"
    elif input_bot == "S" and output_Rule_prev == "P":
            output = output_Rule_prev = "S"
    elif input_bot == "S" and output_Rule_prev == "R":
            output = output_Rule_prev ="R"
    #
    elif input_bot == "P" and output_Rule_prev == "S":
            output = output_Rule_prev ="S"
    elif input_bot == "P" and output_Rule_prev == "P":
            output = output_Rule_prev ="R"
    elif input_bot == "P" and output_Rule_prev == "R":
            output = output_Rule_prev ="P"
    #
    else:
            output = "R" # Just a catch all - should not be used.
    return output

#
############################################################################
# This Bot simply returns a constant output of either Rock, Sissor or Paper.
############################################################################
#
def FixedOutput(input_bot):
    output = "R"
    return output
#
#########################################################
# This simple RSP bot simply returns a random result.
#########################################################
#
def RandomOutput(input_bot):
    output = random.choice(["R","P","S"])
    return output
#
###########################################################
# First Order Markov Chain bot
###########################################################
#
def MarkovFirstOrder(input_bot):
    global random_predictor_FO,markov_model_FO,pair_diff2_FO,pair_diff1_FO
    global output_FO                           #FO implies First Order
#
    if input_bot == '':
# The start up i.e. initialization. This is detected by knowing that there is no input from the bot we are playing against.
        random_predictor_FO = RandomPredictor()
        markov_model_FO = MarkovChain(1, 0.9)  #The Markov chain order and the forgetting factor. 
# Forgetting factor seems to have little impact when the competitor uses a fixed input.
        pair_diff2_FO = ''
        pair_diff1_FO = ''
# further rounds
    else:
        pair_diff2_FO = pair_diff1_FO           #This is the result from the [n-2] game.
        pair_diff1_FO = output_FO + input_bot   #This is our bot's output from the [n-1] game and the competitor bot's decision from the [n-1] game.
#
    if pair_diff2_FO != '':
        markov_model_FO.update_matrix(pair_diff2_FO, input_bot)   #This updates the Markov chain's probability matrix with the [n-2] game results
# and the competitor's bot [n-1] game decision.
#    print("predicter output",markov_model.predict(pair_diff1_FO))
        output_FO = beat[markov_model_FO.predict(pair_diff1_FO)] #This is looking for the maximum probability in the matrix to map the [n-1] game
# Results to estimate the 'best' decision for the nth game (i.e. current game).

    else:
# For the first game in a match the bot simply selects a random output of R, S or P.
# output is a global variable and is the rock sissor paper prediction.
        output_FO = random_predictor_FO.predict() #output is a global variable and is the rock sissor paper prediction.
    return output_FO
#
##############################################################
# Second Order Markov Chain bot
##############################################################
#
def MarkovSecondOrder(input_bot):
#
    global random_predictor_SO,markov_model_SO,pair_diff2_SO,pair_diff1_SO,pair_diff3_SO
    global output_SO,pair_diff_train_SO,pair_diff_predict_SO                           #SO implies Second Order
#
    if input_bot == '':
# The start up i.e. initialization. This is detected by knowing that there is no input from the bot we are playing against.
        random_predictor_SO = RandomPredictor()
        markov_model_SO = MarkovChain(2, 0.9)  #The Markov chain order and the forgetting factor. 
        pair_diff3_SO = ''                     #Major changes from first order Markov Chain are here.
        pair_diff2_SO = ''
        pair_diff1_SO = ''
        pair_diff_train_SO = ''
        pair_diff_predict_SO = ''
# further rounds
    else:
        pair_diff3_SO = pair_diff2_SO          #This is the result from the [n-3] game
        pair_diff2_SO = pair_diff1_SO          #This is the result from the [n-2] game.
        pair_diff1_SO = output_SO + input_bot  #This is our bot's output from the [n-1] game and the competitor bot's decision from the [n-1] game.

#
        pair_diff_train_SO = pair_diff2_SO + pair_diff3_SO   # This is the second order data used to train Markov Model
        pair_diff_predict_SO = pair_diff1_SO + pair_diff2_SO # This is the second order data used to select the best output.
#         
    if pair_diff3_SO != '':
        markov_model_SO.update_matrix(pair_diff_train_SO, input_bot)   #This updates the Markov chain's probability matrix with the [n-2] and [n-3] game results
# and the competitor's bot [n-1] game decision.
#    print("predicter output",markov_model_SO.predict(pair_diff_predict))
        output_SO = beat[markov_model_SO.predict(pair_diff_predict_SO)] #This is looking for the maximum probability in the matrix to map the [n-1] and [n-2] game
#Results to estimate the 'best' decision for the nth game (i.e. current game).

    else:
# For the first game in a match the bot simply selects a random output of R, S or P.
# output is a global variable and is the rock sissor paper prediction.
        output_SO = random_predictor_SO.predict() #output is a global variable and is the rock sissor paper prediction.
    return output_SO
#
#############################################################
# Third Order Markov Chain bot
#############################################################
#
def MarkovThirdOrder(input_bot):
#
    global random_predictor_TO,markov_model_TO,pair_diff2_TO,pair_diff1_TO,pair_diff4_TO,pair_diff3_TO
    global output_TO,pair_diff_train_TO,pair_diff_predict_TO                          #TO implies Third Order
#
    if input_bot == '':
# The start up i.e. initialization. This is detected by knowing that there is no input from the bot we are playing against.
        random_predictor_TO = RandomPredictor()
        markov_model_TO = MarkovChain(3, 0.9)  #The Markov chain order and the forgetting factor. 
        pair_diff4_TO = ''
        pair_diff3_TO = ''                     #Major changes from first and second order Markov Chain is here.
        pair_diff2_TO = ''
        pair_diff1_TO = ''
        pair_diff_train_TO = ''
        pair_diff_predict_TO = ''        
# further rounds
    else:
        pair_diff4_TO = pair_diff3_TO          #This is the result from the [n-4] game
        pair_diff3_TO = pair_diff2_TO          #This is the result from the [n-3] game
        pair_diff2_TO = pair_diff1_TO          #This is the result from the [n-2] game.
        pair_diff1_TO = output_TO + input_bot  #This is our bot's output from the [n-1] game and the competitor bot's decision from the [n-1] game.
#
        pair_diff_train_TO = pair_diff2_TO + pair_diff3_TO + pair_diff4_TO   # This is the third order data used to train Markov Model
        pair_diff_predict_TO = pair_diff1_TO + pair_diff2_TO + pair_diff3_TO # This is the third order data used to select the best output.
#
    if pair_diff4_TO != '':
        markov_model_TO.update_matrix(pair_diff_train_TO, input_bot)   #This updates the Markov chain's probability matrix with the [n-2],[n-3] and [n-4] game results
# and the competitor's bot [n-1] game decision.
#    print("predicter output",markov_model_TO.predict(pair_diff_predict))
        output_TO = beat[markov_model_TO.predict(pair_diff_predict_TO)] #This is looking for the maximum probability in the matrix to map the [n-1], [n-2] and [n-3] game
# Results to estimate the 'best' decision for the nth game (i.e. current game).
    else:
# For the first game in a match the bot simply selects a random output of R, S or P.
# output is a global variable and is the rock sissor paper prediction.
        output_TO = random_predictor_TO.predict() #output is a global variable and is the rock sissor paper prediction.
    return output_TO
#
############################################################
# Fourth Order Markov Chain bot
############################################################
#
def MarkovFourthOrder(input_bot):
#
    global random_predictor_FRO,markov_model_FRO,pair_diff2_FRO,pair_diff1_FRO,pair_diff5_FRO, pair_diff4_FRO,pair_diff3_FRO
    global output_FRO,pair_diff_train_FRO,pair_diff_predict_FRO                         #FRO implies Fourth Order
#
    if input_bot == '':
# The start up i.e. initialization. This is detected by knowing that there is no input from the bot we are playing against.
        random_predictor_FRO = RandomPredictor()
        markov_model_FRO = MarkovChain(4, 0.9)  #The Markov chain order and the forgetting factor.
        pair_diff5_FRO = ''
        pair_diff4_FRO = ''
        pair_diff3_FRO = ''                     #Major changes from first, second and third order Markov Chain is here.
        pair_diff2_FRO = ''
        pair_diff1_FRO = ''
        pair_diff_train_FRO =''
        pair_diff_predict_FRO =''
# further rounds
    else:
        pair_diff5_FRO = pair_diff4_FRO         #This is the result from the [n-5] game        
        pair_diff4_FRO = pair_diff3_FRO         #This is the result from the [n-4] game
        pair_diff3_FRO = pair_diff2_FRO         #This is the result from the [n-3] game
        pair_diff2_FRO = pair_diff1_FRO         #This is the result from the [n-2] game.
        pair_diff1_FRO = output_FRO + input_bot #This is our bot's output from the [n-1] game and the competitor bot's decision from the [n-1] game.
#
        pair_diff_train_FRO = pair_diff2_FRO + pair_diff3_FRO + pair_diff4_FRO + pair_diff5_FRO    # This is the fourth order data used to train Markov Model
        pair_diff_predict_FRO = pair_diff1_FRO + pair_diff2_FRO + pair_diff3_FRO + pair_diff4_FRO  # This is the fourth order data used to select the best output.
#
    if pair_diff5_FRO != '':
        markov_model_FRO.update_matrix(pair_diff_train_FRO, input_bot)   #This updates the Markov chain's probability matrix with the [n-2],[n-3], [n-4] and [n-5] game results
# and the competitor's bot [n-1] game decision.
#    print("predicter output",markov_model_TO.predict(pair_diff_predict))
        output_FRO = beat[markov_model_FRO.predict(pair_diff_predict_FRO)] #This is looking for the maximum probability in the matrix to map the [n-1], [n-2], [n-3] and [n-4] game
# Results to estimate the 'best' decision for the nth game (i.e. current game).
    else:
# For the first game in a match the bot simply selects a random output of R, S or P.
# output is a global variable and is the rock sissor paper prediction.
        output_FRO = random_predictor_FRO.predict() #output is a global variable and is the rock sissor paper prediction.
    return output_FRO
#
####################################################
# Fifth Order Markov Chain bot
####################################################
#
def MarkovFifthOrder(input_bot):
#
    global random_predictor_FHO,markov_model_FHO,pair_diff2_FHO,pair_diff1_FHO,pair_diff6_FHO, pair_diff5_FHO, pair_diff4_FHO,pair_diff3_FHO
    global output_FHO,pair_diff_train_FHO,pair_diff_predict_FHO                           #FHO implies Fith Order
#
    if input_bot == '':
# The start up i.e. initialization. This is detected by knowing that there is no input from the bot we are playing against.
        random_predictor_FHO = RandomPredictor()
        markov_model_FHO = MarkovChain(5, 0.9)  #The Markov chain order and the forgetting factor.
        pair_diff6_FHO =''                      
        pair_diff5_FHO = ''                     #Major changes from first, second, third and fourth order Markov Chain is here.
        pair_diff4_FHO = ''
        pair_diff3_FHO = ''                    
        pair_diff2_FHO = ''
        pair_diff1_FHO = ''
        pair_diff_train_FHO = ''
        pair_diff_predict_FHO = ''        
# further rounds
    else:
        pair_diff6_FHO = pair_diff5_FHO         #This is the result from the [n-6] game
        pair_diff5_FHO = pair_diff4_FHO         #This is the result from the [n-5] game        
        pair_diff4_FHO = pair_diff3_FHO         #This is the result from the [n-4] game
        pair_diff3_FHO = pair_diff2_FHO         #This is the result from the [n-3] game
        pair_diff2_FHO = pair_diff1_FHO         #This is the result from the [n-2] game.
        pair_diff1_FHO = output_FHO + input_bot #This is our bot's output from the [n-1] game and the competitor bot's decision from the [n-1] game.
#
        pair_diff_train_FHO = pair_diff2_FHO + pair_diff3_FHO + pair_diff4_FHO + pair_diff5_FHO + pair_diff6_FHO    # This is the fifth order data used to train Markov Model
        pair_diff_predict_FHO = pair_diff1_FHO + pair_diff2_FHO + pair_diff3_FHO + pair_diff4_FHO + pair_diff5_FHO  # This is the fifth order data used to select the best output.
#
    if pair_diff6_FHO != '':
        markov_model_FHO.update_matrix(pair_diff_train_FHO, input_bot)   #This updates the Markov chain's probability matrix with the [n-2],[n-3], [n-4], [n-5] and [n-6] game results
# and the competitor's bot [n-1] game decision.
#    print("predicter output",markov_model_TO.predict(pair_diff_predict))
        output_FHO = beat[markov_model_FHO.predict(pair_diff_predict_FHO)] #This is looking for the maximum probability in the matrix to map the [n-1], [n-2], [n-3], [n-4] and [n-5] game
# results to estimate the 'best' decision for the nth game (i.e. current game).
    else:
# For the first game in a match the bot simply selects a random output of R, S or P.
# output is a global variable and is the rock sissor paper prediction.
        output_FHO = random_predictor_FHO.predict() #output is a global variable and is the rock sissor paper prediction.
    return output_FHO

####################################################################################################################
# This bot combines a first through to fifth order Markov chain. The five Markov Chains
# run in parrallel and the output selected dependant on which one wins consistently.
# 
# The global parameter naming convention is different to first, second, third, fourth and fifth order Markov Chains.
# This avoids conflict between different AI bots.
####################################################################################################################
#
def MarkovMultiOrderSwitch(input_bot):
#
    global random_predictor,markov_model,random_predictor_second,markov_model_second
    global random_predictor_third,markov_model_third,random_predictor_fourth, markov_model_fourth, random_predictor_fifth, markov_model_fifth 
    global output_first, pair_diff2,pair_diff1
    global output_second, pair_diff3_second,pair_diff2_second,pair_diff1_second,pair_diff_train_second,pair_diff_predict_second
    global output_third, pair_diff4_third, pair_diff3_third,pair_diff2_third,pair_diff1_third,pair_diff_train_third,pair_diff_predict_third
    global output_fourth,pair_diff5_fourth, pair_diff4_fourth, pair_diff3_fourth,pair_diff2_fourth,pair_diff1_fourth,pair_diff_train_fourth,pair_diff_predict_fourth
    global output_fifth,pair_diff6_fifth,pair_diff5_fifth, pair_diff4_fifth, pair_diff3_fifth,pair_diff2_fifth,pair_diff1_fifth,pair_diff_train_fifth,pair_diff_predict_fifth
    global count_first_order, count_second_order, count_third_order, count_fourth_order, count_fifth_order
#    
    if input_bot == '':
# Counter initialization
        count_first_order = 0
        count_second_order = 0
        count_third_order = 0
        count_fourth_order = 0
        count_fifth_order =0
# The First Order initialization. This is detected by knowing that there is no input_bot from the bot we are playing against.
        random_predictor = RandomPredictor()
        markov_model = MarkovChain(1, 0.9)  #The Markov chain order and the forgetting factor. 
# Forgetting factor seems to have little impact when the competitor uses a fixed input.
        pair_diff2 = ''
        pair_diff1 = ''   
# Markov Second Order initialization.  
        random_predictor_second = RandomPredictor()
        markov_model_second = MarkovChain(2, 0.9)  # The Markov chain order and the forgetting factor.
        pair_diff3_second = ''                     # Major changes from first order Markov Chain is here.
        pair_diff2_second = ''
        pair_diff1_second = ''
        pair_diff_train_second = ''
        pair_diff_predict_second = ''
# Markov Third Order initialization.  
        random_predictor_third = RandomPredictor()
        markov_model_third = MarkovChain(3, 0.9)  # The Markov chain order and the forgetting factor.
        pair_diff4_third = '' 
        pair_diff3_third = ''                     # Major changes from second order Markov Chain is here.
        pair_diff2_third = ''
        pair_diff1_third = ''
        pair_diff_train_third = ''
        pair_diff_predict_third = ''
# Markov Fourth Order initialization.  
        random_predictor_fourth = RandomPredictor()
        markov_model_fourth = MarkovChain(4, 0.9)  # The Markov chain order and the forgetting factor.
        pair_diff5_fourth = ''
        pair_diff4_fourth = '' 
        pair_diff3_fourth = ''                     # Major changes from third order Markov Chain is here.
        pair_diff2_fourth = ''
        pair_diff1_fourth = ''
        pair_diff_train_fourth = ''
        pair_diff_predict_fourth = ''
# Markov Fifth Order initialization.  
        random_predictor_fifth = RandomPredictor()
        markov_model_fifth = MarkovChain(5, 0.9)  # The Markov chain order and the forgetting factor.
        pair_diff6_fifth = ''
        pair_diff5_fifth = ''
        pair_diff4_fifth = ''
        pair_diff3_fifth = ''                     # Major changes from fourth order Markov Chain is here.
        pair_diff2_fifth = ''
        pair_diff1_fifth = ''
        pair_diff_train_fifth = ''
        pair_diff_predict_fifth = ''
# further rounds
    else:
#First Order Markov Chain Update
        pair_diff2 = pair_diff1                         #This is the result from the [n-2] game.
        pair_diff1 = output_first + input_bot           #This is our bot's output from the [n-1] game and the competitor bot's decision from the [n-1] game.
#Second Order Markov Chain Update
        pair_diff3_second = pair_diff2_second           #This is the result from the [n-3] game
        pair_diff2_second = pair_diff1_second           #This is the result from the [n-2] game.
        pair_diff1_second = output_second + input_bot   #This is our bot's output from the [n-1] game and the competitor bot's decision from the [n-1] game.
#
        pair_diff_train_second = pair_diff2_second + pair_diff3_second   # This is the second order data used to train Markov Model
        pair_diff_predict_second = pair_diff1_second + pair_diff2_second # This is the second order data used to select the best output.
#
#Third Order Markov Chain Update
        pair_diff4_third = pair_diff3_third             #This is the result from the [n-4] game
        pair_diff3_third = pair_diff2_third             #This is the result from the [n-3] game
        pair_diff2_third = pair_diff1_third             #This is the result from the [n-2] game.
        pair_diff1_third = output_third + input_bot     #This is our bot's output from the [n-1] game and the competitor bot's decision from the [n-1] game.
#
        pair_diff_train_third = pair_diff2_third + pair_diff3_third + pair_diff4_third   # This is the third order data used to train Markov Model
        pair_diff_predict_third = pair_diff1_third + pair_diff2_third + pair_diff3_third # This is the third order data used to select the best output.
#
#Fourth Order Markov Chain Update
        pair_diff5_fourth = pair_diff4_fourth           #This is the result from the [n-5] game
        pair_diff4_fourth = pair_diff3_fourth           #This is the result from the [n-4] game
        pair_diff3_fourth = pair_diff2_fourth           #This is the result from the [n-3] game
        pair_diff2_fourth = pair_diff1_fourth           #This is the result from the [n-2] game.
        pair_diff1_fourth = output_fourth + input_bot   #This is our bot's output from the [n-1] game and the competitor bot's decision from the [n-1] game.
#
        pair_diff_train_fourth = pair_diff2_fourth + pair_diff3_fourth + pair_diff4_fourth + pair_diff5_fourth   # This is the fourth order data used to train Markov Model
        pair_diff_predict_fourth = pair_diff1_fourth + pair_diff2_fourth + pair_diff3_fourth + pair_diff4_fourth # This is the fourth order data used to select the best output.
#
#Fifth Order Markov Chain Update
        pair_diff6_fifth = pair_diff5_fifth            #This is the result from the [n-6] game
        pair_diff5_fifth = pair_diff4_fifth            #This is the result from the [n-5] game
        pair_diff4_fifth = pair_diff3_fifth            #This is the result from the [n-4] game
        pair_diff3_fifth = pair_diff2_fifth            #This is the result from the [n-3] game
        pair_diff2_fifth = pair_diff1_fifth            #This is the result from the [n-2] game.
        pair_diff1_fifth = output_fifth + input_bot    #This is our bot's output from the [n-1] game and the competitor bot's decision from the [n-1] game.
#
        pair_diff_train_fifth = pair_diff2_fifth + pair_diff3_fifth + pair_diff4_fifth + pair_diff5_fifth + pair_diff6_fifth    # This is the fifth order data used to train Markov Model
        pair_diff_predict_fifth = pair_diff1_fifth + pair_diff2_fifth + pair_diff3_fifth + pair_diff4_fifth + pair_diff5_fifth  # This is the fifth order data used to select the best output.
#
# First Order Markov Chain Update
#
    if pair_diff2 != '':
        markov_model.update_matrix(pair_diff2, input_bot)   #This updates the Markov chain's probability matrix with the [n-2] game results
# and the competitor's bot [n-1] game decision.
#    print("predicter output",markov_model.predict(pair_diff1))
        output_first = beat[markov_model.predict(pair_diff1)] #This is looking for the maximum probability in the matrix to map the [n-1] game
# results to estimate the 'best' decision for the nth game (i.e. current game).
    else:
# For the first game in a match the bot simply selects a random output of R, S or P.
# output is a global variable and is the rock sissor paper prediction.
        output_first = random_predictor.predict() #output is a global variable and is the rock sissor paper prediction.
#        
# Second Order Markov Chain Update.
#
    if pair_diff3_second != '':
        markov_model_second.update_matrix(pair_diff_train_second, input_bot)   #This updates the Markov chain's probability matrix with the [n-2] and [n-3] game results
# and the competitor's bot [n-1] game decision.
#    print("predicter output",markov_model.predict(pair_diff_predict))
        output_second = beat[markov_model_second.predict(pair_diff_predict_second)] #This is looking for the maximum probability in the matrix to map the [n-1] and [n-2] game
# results to estimate the 'best' decision for the nth game (i.e. current game).
    else:
# For the first game in a match the bot simply selects a random output of R, S or P.
# output is a global variable and is the rock sissor paper prediction.
        output_second = random_predictor_second.predict() #output is a global variable and is the rock sissor paper prediction.
#        
# Third Order Markov Chain Update.
#
    if pair_diff4_third != '':
        markov_model_third.update_matrix(pair_diff_train_third, input_bot)   #This updates the Markov chain's probability matrix with the [n-2], [n-3] and [n-4] game results
# and the competitor's bot [n-1] game decision.
#    print("predicter output",markov_model.predict(pair_diff_predict))
        output_third = beat[markov_model_third.predict(pair_diff_predict_third)] #This is looking for the maximum probability in the matrix to map the [n-1], [n-2] and [n-3] game
# results to estimate the 'best' decision for the nth game (i.e. current game).
    else:
# For the first game in a match the bot simply selects a random output of R, S or P.
# output is a global variable and is the rock sissor paper prediction.
        output_third = random_predictor_third.predict() #output is a global variable and is the rock sissor paper prediction.
#        
# Fourth Order Markov Chain Update.
#
    if pair_diff5_fourth != '':
        markov_model_fourth.update_matrix(pair_diff_train_fourth, input_bot)   #This updates the Markov chain's probability matrix with the [n-2], [n-3], [n-4] and [n-5] game results
# and the competitor's bot [n-1] game decision.
#    print("predicter output",markov_model.predict(pair_diff_predict))
        output_fourth = beat[markov_model_fourth.predict(pair_diff_predict_fourth)] #This is looking for the maximum probability in the matrix to map the [n-1] and [n-2] game
# results to estimate the 'best' decision for the nth game (i.e. current game).
    else:
# For the first game in a match the bot simply selects a random output of R, S or P.
# output is a global variable and is the rock sissor paper prediction.
        output_fourth = random_predictor_fourth.predict() #output is a global variable and is the rock sissor paper prediction.
#        
# Fifth Order Markov Chain Update.
#
    if pair_diff6_fifth != '':
        markov_model_fifth.update_matrix(pair_diff_train_fifth, input_bot)   #This updates the Markov chain's probability matrix with the [n-2], [n-3], [n-5] and [n-6] game results
# and the competitor's bot [n-1] game decision.
#    print("predicter output",markov_model.predict(pair_diff_predict))
        output_fifth = beat[markov_model_fifth.predict(pair_diff_predict_fifth)] #This is looking for the maximum probability in the matrix to map the [n-1] and [n-2] game
# results to estimate the 'best' decision for the nth game (i.e. current game).
    else:
# For the first game in a match the bot simply selects a random output of R, S or P.
# output is a global variable and is the rock sissor paper prediction.
        output_fifth = random_predictor_fifth.predict() #output is a global variable and is the rock sissor paper prediction.
#
# Decision to play either output from First through to Fifth Order Markov model
#
    if pair_diff1 =="SP" or pair_diff1=="PR" or pair_diff1 =="RS":                          #First order Markov is the default as most reactive due to limited memory.
        count_first_order += 1
        output = output_first
#
    elif pair_diff1_second =="SP" or pair_diff1_second =="PR" or pair_diff1_second =="RS":  #If second order Markov wins then use it as more intelligent than first order.
        count_second_order += 1
        output = output_second
#
    elif pair_diff1_third =="SP" or pair_diff1_third =="PR" or pair_diff1_third =="RS":     #If third order Markov is wins then use it as more intelligent than lower orders
        count_third_order += 1
        output = output_third
#
    elif pair_diff1_fourth =="SP" or pair_diff1_fourth =="PR" or pair_diff1_fourth =="RS":  #If fourth order Markov wins then use it as more intelligent than lower orders.
        count_fourth_order += 1
        output = output_fourth
#
    elif pair_diff1_fifth =="SP" or pair_diff1_fifth =="PR" or pair_diff1_fifth =="RS":     #If fourth order Markov wins then use it as more intelligent than lower orders.
        count_fifth_order += 1
        output = output_fifth
#
# If both higher order loose or draw then always use first order as more reactive.
# Could do better with this logic?
    else:
        output = output_first 
    return output

#
##########################################################################################
# THE MAIN CALLING ROUTINE THAT SCHEDULES THE TWO BOTS AND PLAYS THEM AGAINST EACH OTHER
#
# IMPORTANT: YOU CANN'T RUN TWO AIs OF THE SAME NAME AGAINST EACH OTHER AS THEY DEFINE GLOBAL
# VARIABLES INSIDE THE FUNCTION THAT WILL OVERWRITE EACH OTHER.
#########################################################################################
#
# Play AI and computer bots against each other
#
def main():                                 
#Initializations
    while True:
        output_AIbot_prev = ""
        output_opponent_prev = ""
        output_AIbot = ""
        output_opponent = ""
        AIbot_wins =0
        opponentbot_wins =0
        draws = 0
        number_games = ""
        order = ""
        play_again = ""
        #winrate = (opponentbot_wins/number_games)*100
        #lossrate = (AIbot_wins/number_games)*100
        #drawrate = (draws/number_games)*100
        print("Hello and Welcome!")
        print("This is my code demostration for AI (that being Markov chains) playing Rock, Paper, Scissors (RPS).")
        print("")
        while True:
            try:
                number_games = int(input("Enter a positive integer between 1 to 10000 for the number of RPS games to play: "))
                assert 0 < number_games < 10001
            except ValueError:
                print("Incorrect input - not a positive integer. Please try again.")
                print("")
            except AssertionError:
                print("Incorrect input - not a positive integer between 1 to 10000. Please try again.")
                print("")
            else:
                break

        while True:        
            order = input("Enter the Markov chain Order to play against the Variable Order Markov chain (Type either 'first', 'second', 'third', 'fourth' or 'fifth'): ").lower()
            if order == "first":
                for numgames in range(1,number_games+1):
                    output_opponent = MarkovFirstOrder(output_AIbot_prev)                 
                    output_AIbot = MarkovMultiOrderSwitch(output_opponent_prev)             

                    result = output_AIbot+output_opponent
                    if result == "RS" or result == "PR" or result == "SP":
                        AIbot_wins +=1
                    elif result == "SR" or result == "RP" or result == "PS":
                        opponentbot_wins +=1
                    else:
                        draws +=1       

                    output_AIbot_prev=output_AIbot
                    output_opponent_prev=output_opponent
                print("Number of First Order Markov chain Wins: ",opponentbot_wins)
                print("Number of Variable Order Markov chain Wins: ",AIbot_wins)
                print("Number of Draws: ",draws)
                #print("First Order Markov Chain AI winrate: ",winrate)
                #print("First Order Markov Chain AI loss rate: ",lossrate)
                #print("First Order Markov Chain AI draw rate: ",drawrate)
                print("")
                break

            elif order == "second":
                for numgames in range(1,number_games+1):
                    output_opponent = MarkovSecondOrder(output_AIbot_prev)                   
                    output_AIbot = MarkovMultiOrderSwitch(output_opponent_prev)             

                    result = output_AIbot+output_opponent
                    if result == "RS" or result == "PR" or result == "SP":
                        AIbot_wins +=1
                    elif result == "SR" or result == "RP" or result == "PS":
                        opponentbot_wins +=1
                    else:
                        draws +=1       

                    output_AIbot_prev=output_AIbot
                    output_opponent_prev=output_opponent
                print("Number of Second Order Markov chain Wins: ",opponentbot_wins)
                print("Number of Variable Order Markov chain Wins: ",AIbot_wins)
                print("Number of Draws: ",draws)
                print("")
                break

            elif order == "third":
                for numgames in range(1,number_games+1):
                    output_opponent = MarkovThirdOrder(output_AIbot_prev)                 
                    output_AIbot = MarkovMultiOrderSwitch(output_opponent_prev)              

                    result = output_AIbot+output_opponent
                    if result == "RS" or result == "PR" or result == "SP":
                        AIbot_wins +=1
                    elif result == "SR" or result == "RP" or result == "PS":
                        opponentbot_wins +=1
                    else:
                        draws +=1       

                    output_AIbot_prev=output_AIbot
                    output_opponent_prev=output_opponent
                print("Number of Third Order Markov chain Wins: ",opponentbot_wins)
                print("Number of Variable Order Markov chain Wins: ",AIbot_wins)
                print("Number of Draws: ",draws)
                print("")
                break

            elif order == "fourth":
                for numgames in range(1,number_games+1):
                    output_opponent = MarkovThirdOrder(output_AIbot_prev)                  
                    output_AIbot = MarkovMultiOrderSwitch(output_opponent_prev)           
                    result = output_AIbot+output_opponent
                    if result == "RS" or result == "PR" or result == "SP":
                        AIbot_wins +=1
                    elif result == "SR" or result == "RP" or result == "PS":
                        opponentbot_wins +=1
                    else:
                        draws +=1       

                    output_AIbot_prev=output_AIbot
                    output_opponent_prev=output_opponent
                print("Number of Fourth Order Markov chain Wins: ",opponentbot_wins)
                print("Number of Variable Order Markov chain Wins: ",AIbot_wins)
                print("Number of Draws: ",draws)
                print("")
                break

            elif order == "fifth":
                for numgames in range(1,number_games+1):
                    output_opponent = MarkovThirdOrder(output_AIbot_prev)                  
                    output_AIbot = MarkovMultiOrderSwitch(output_opponent_prev)             

                    result = output_AIbot+output_opponent
                    if result == "RS" or result == "PR" or result == "SP":
                        AIbot_wins +=1
                    elif result == "SR" or result == "RP" or result == "PS":
                        opponentbot_wins +=1
                    else:
                        draws +=1       

                    output_AIbot_prev=output_AIbot
                    output_opponent_prev=output_opponent
                print("Number of Fifth Order Markov chain Wins: ",opponentbot_wins)
                print("Number of Variable Order Markov chain Wins: ",AIbot_wins)
                print("Number of Draws: ",draws)
                print("")
                break

            else:
                print("Incorrect input - Please try again.")
                print("")
        
        play_again = input("Do you want to play again (Type either 'continue' or 'quit'): ").lower()
        if play_again == "continue":
            print("Rerunning...")
            print("")
        elif play_again == "quit":
            print("Thank you for playing. Goodbye!")
            return
   
            
        
            
        
            

        
##########################################################################
"""
    for numgames in range(1,number_games):
#
# Opponent bots
#

        output_opponent = MarkovFirstOrder(output_AIbot_prev)                  #Tested okay 
#        output_opponent = MarkovSecondOrder(output_AIbot_prev)                 #Tested okay
#        output_opponent = MarkovThirdOrder(output_AIbot_prev)                  #Tested okay
#        output_opponent = MarkovFourthOrder(output_AIbot_prev)                 #Tested okay
#        output_opponent = MarkovFifthOrder(output_AIbot_prev)                 #Tested okay
        output_AIbot = MarkovMultiOrderSwitch(output_opponent_prev)             #Tested okay 

#
# Calculate the winning bot
#
        result = output_AIbot+output_opponent
        if result == "RS" or result == "PR" or result == "SP":
            AIbot_wins +=1
        elif result == "SR" or result == "RP" or result == "PS":
            opponentbot_wins +=1
        else:
            draws +=1       
# Setup outputs and inputs for next game        
        output_AIbot_prev=output_AIbot
        output_opponent_prev=output_opponent


    print("Number of AIBot wins: ",AIbot_wins)
    print("Number of Opponent Bot wins: ",opponentbot_wins)
    print("Difference: ",AIbot_wins-opponentbot_wins)
    print("Number of draws: ",draws)
    print("")
    return
"""
##########################################################################

    


#
############################################################################################################
# Running MAIN Routine
############################################################################################################
#
main()




Hello and Welcome!
This is my code demostration for AI (that being Markov chains) playing Rock, Paper, Scissors (RPS).



Enter a positive integer between 1 to 10000 for the number of RPS games to play:  10000
Enter the Markov chain Order to play against the Variable Order Markov chain (Type either 'first', 'second', 'third', 'fourth' or 'fifth'):  first


Number of First Order Markov chain Wins:  3855
Number of Variable Order Markov chain Wins:  4379
Number of Draws:  1766



Do you want to play again (Type either 'continue' or 'quit'):  continue


Rerunning...

Hello and Welcome!
This is my code demostration for AI (that being Markov chains) playing Rock, Paper, Scissors (RPS).



Enter a positive integer between 1 to 10000 for the number of RPS games to play:  10000
Enter the Markov chain Order to play against the Variable Order Markov chain (Type either 'first', 'second', 'third', 'fourth' or 'fifth'):  first


Number of First Order Markov chain Wins:  3896
Number of Variable Order Markov chain Wins:  4417
Number of Draws:  1687

